# *ml-1D-distributed-model-runs.ipynb*

# Running models in parallel

This is a work-in-progress-notebook. Currently, it allows us to: specify a models bank for 160 Random Forest and Extra Trees models; assemble the datasets for different folds of year-wise cross-validation; run the models for 12 combinations of region, crop, compositing scheme, and in-season date; and record results (accuracies and uncertainties) in csv files.

Function definitions at the top of the notebook; then sequential model runs; then parallel model runs.

New in version 1D: parallel model runs using Dask.

New in version 1C: can draw from bank of specified models, and run through them in a for loop.

New in version 1B: can now specify `in_season`, which is None (meaning use whole year), or 230 (thru DOY 230 which is mid-August), or 160 (thru DOY 160 which is early June)

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
def create_models_bank():
    models_bank = {}
    increment = 0
    for n_estimators in [200, 500]:
        for max_features in [0.05, 0.1, 0.2, 0.4, 1.0]:
            for min_samples_split in [2, 4]:
                for bootstrap in [False, True]:
                    for class_weight in [None, 'balanced']:

                        increment += 1

                        three_digit = str(increment).zfill(3)
                        models_bank[three_digit] = {
                            'n_estimators': n_estimators,
                            'max_features': max_features,
                            'bootstrap': bootstrap,
                            'min_samples_split': min_samples_split,
                            'class_weight': class_weight,
                            'n_jobs': -1
                        }

    return models_bank

In [3]:
def create_X_y_single_year(tile,year,scheme_name,crop_of_interest_id,
                          in_season=None):
    coiid = crop_of_interest_id
    
    refl = np.load(f'../data/composited_interpolated/Refl_{tile}_{year}_{scheme_name}.npy')

    # Determine nrf (number of reflectance features)
    if in_season not in [160, 230, None]:
        print('Please change in_season to 160, 230, or None.')
        return None
    if in_season is None:
        nrf = refl.shape[1]
    if in_season in [160, 230]:
        pdsize = int(scheme_name[:-3])
        nrf = ((in_season - 90) // pdsize) * 6 + 6
        
    crop = []
    for y in range(year-4,year+1):
        crop.append(np.load(f'../data/processed_crop/Crop_{tile}_{y}.npy')==coiid)

    X = np.column_stack([refl[:,:nrf]] + crop[:-1])
    y = crop[-1]
    
    return X, y

In [4]:
def create_X_y_multiyear(tile,
                      years,
                      scheme_name,
                      crop_of_interest_id,
                        in_season=None):
    X_list = []
    y_list = []
    
    for y in years:
        X, y = create_X_y_single_year(tile,y,scheme_name,
                                      crop_of_interest_id,
                                     in_season)
        X_list.append(X)
        y_list.append(y)
    
    X = np.concatenate(X_list)
    y = np.concatenate(y_list)
    
    return X, y

In [5]:
def create_X_y(tile,
              years,
              scheme_name,
              crop_of_interest_id,
              in_season=None
              ):
    if type(years)==int:
        return create_X_y_single_year(tile,
                                      years,
                                      scheme_name,
                                      crop_of_interest_id,
                                      in_season)
    
    return create_X_y_multiyear(tile,
                              years,
                              scheme_name,
                              crop_of_interest_id,
                              in_season)

In [6]:
def fit_predict_report(model_name,
                      model,
                      training_sample_size,
                      validation_sample_size,
                      tile,
                      years,
                      scheme_name,
                      crop_of_interest_id,
                      in_season
                      ):
    
    # produce csv_name
    exempt = ['years', 'model']
    param_value_strings = [f'{model_name}',
                      f'{training_sample_size}',
                      f'{validation_sample_size}',
                      f'{tile}',
                      f'{scheme_name}',
                      f'{crop_of_interest_id}',
                      f'{in_season}']
    csv_name = '_'.join(param_value_strings) +'.csv'

    # check whether previously run and, if so, end the effort
    if csv_name in os.listdir('../data/results/'):
        return 'If you see this, the specified model was previously run.'

    print(f'-- Process for {csv_name} --')
    
    # below is actually fitting and predicting and reporting
    
    conf = []

    for val_year in years:
        print('Starting a fold...')
        print('> Assembling the datasets')
        train_years = [yr for yr in range(2018,2023) if yr!=val_year]

        X_train0, y_train0 = create_X_y(tile,train_years,
                                      scheme_name,crop_of_interest_id,
                                       in_season)
        X_val0, y_val0 = create_X_y(tile,val_year,
                                  scheme_name,crop_of_interest_id,
                                  in_season)

        if training_sample_size is not None:
            X_train, X_trsurplus, y_train, y_trsurplus = train_test_split(X_train0,
                                                                     y_train0,
                                                                     train_size=training_sample_size,
                                                                     random_state=19)
        if training_sample_size is None:
            X_train, y_train = X_train0, y_train0

        if validation_sample_size is not None:
            X_val, X_vsurplus, y_val, y_vsurplus = train_test_split(X_val0,
                                                                     y_val0,
                                                                     train_size=validation_sample_size,
                                                                     random_state=19)
        if validation_sample_size is None:
            X_val, y_val = X_val0, y_val0
    
            
        print('> Fitting the model on the training set')
        model.fit(X_train, y_train)
        print('> Predicting on the validation set')
        pred = model.predict(X_val)

        print('> Recording performance metrics')
        act = y_val
        ActPred_00 = sum((act==0) & (pred==0))
        ActPred_01 = sum((act==0) & (pred==1))
        ActPred_10 = sum((act==1) & (pred==0))
        ActPred_11 = sum((act==1) & (pred==1))
        conf_1yr = [ActPred_00, ActPred_01, ActPred_10, ActPred_11]

        conf.append(conf_1yr)
        print('Finished a fold.')

    carr = np.array(conf)

    carr = np.row_stack([carr,np.full((2,4),-1)])

    # above we added the totals row
    # now we need to add the columns for precision and recall

    # create dataframe
    cdf = pd.DataFrame(data = carr,
                      index = [f'ValYear{yr}' for yr in years]+['Mean','StdE'],
                      columns = ['ActPred_00', 'ActPred_01', 
                                 'ActPred_10', 'ActPred_11']
                      )

    cdf['Precision'] = cdf.ActPred_11 / (cdf.ActPred_01 + cdf.ActPred_11)
    cdf['Recall'] = cdf.ActPred_11 / (cdf.ActPred_10 + cdf.ActPred_11)
    cdf['F1'] = 2*cdf.Precision*cdf.Recall / (cdf.Precision + cdf.Recall)
    for col in ['Precision','Recall','F1']:
        cdf.at['Mean',col] = np.mean(cdf.loc[:'ValYear2022',col])
        cdf.at['StdE',col] = np.std(cdf.loc[:'ValYear2022',col])
    
    
    param_strings = [f'# model_name: {model_name}',
                     f'# model: {model}',
                      f'# training_sample_size: {training_sample_size}',
                      f'# validation_sample_size: {validation_sample_size}',
                      f'# tile: {tile}',
                      f'# scheme_name: {scheme_name}',
                      f'# crop_of_interest_id: {crop_of_interest_id}',
                      f'# in_season: {in_season}']
    comment = '\n'.join(param_strings) + '\n' 
    with open(f'../data/results/{csv_name}', 'a') as f:
        f.write(comment)
        cdf.to_csv(f)
    
    print(f'Find results in ../data/results/{csv_name}')
    
    return f'Find results in ../data/results/{csv_name}'

In [ ]:
import ctypes
def trim_memory() -> int:
     libc = ctypes.CDLL("libc.so.6")
     return libc.malloc_trim(0)

In [ ]:
def trim_fit_predict_report(model_name,
                      model,
                      training_sample_size,
                      validation_sample_size,
                      tile,
                      years,
                      scheme_name,
                      crop_of_interest_id,
                      in_season#,
                      #client
                      ):
    client.run(trim_memory)
    
    # produce csv_name
    exempt = ['years', 'model']
    param_value_strings = [f'{model_name}',
                      f'{training_sample_size}',
                      f'{validation_sample_size}',
                      f'{tile}',
                      f'{scheme_name}',
                      f'{crop_of_interest_id}',
                      f'{in_season}']
    csv_name = '_'.join(param_value_strings) +'.csv'

    # check whether previously run and, if so, end the effort
    if csv_name in os.listdir('../data/results/'):
        return 'If you see this, the specified model was previously run.'

    print(f'-- Process for {csv_name} --')
    
    # below is actually fitting and predicting and reporting
    
    conf = []

    for val_year in years:
        print('Starting a fold...')
        print('> Assembling the datasets')
        train_years = [yr for yr in range(2018,2023) if yr!=val_year]

        X_train0, y_train0 = create_X_y(tile,train_years,
                                      scheme_name,crop_of_interest_id,
                                       in_season)
        X_val0, y_val0 = create_X_y(tile,val_year,
                                  scheme_name,crop_of_interest_id,
                                  in_season)

        if training_sample_size is not None:
            X_train, X_trsurplus, y_train, y_trsurplus = train_test_split(X_train0,
                                                                     y_train0,
                                                                     train_size=training_sample_size,
                                                                     random_state=19)
        if training_sample_size is None:
            X_train, y_train = X_train0, y_train0

        if validation_sample_size is not None:
            X_val, X_vsurplus, y_val, y_vsurplus = train_test_split(X_val0,
                                                                     y_val0,
                                                                     train_size=validation_sample_size,
                                                                     random_state=19)
        if validation_sample_size is None:
            X_val, y_val = X_val0, y_val0
    
            
        print('> Fitting the model on the training set')
        model.fit(X_train, y_train)
        print('> Predicting on the validation set')
        pred = model.predict(X_val)

        print('> Recording performance metrics')
        act = y_val
        ActPred_00 = sum((act==0) & (pred==0))
        ActPred_01 = sum((act==0) & (pred==1))
        ActPred_10 = sum((act==1) & (pred==0))
        ActPred_11 = sum((act==1) & (pred==1))
        conf_1yr = [ActPred_00, ActPred_01, ActPred_10, ActPred_11]

        conf.append(conf_1yr)
        print('Finished a fold.')

    carr = np.array(conf)

    carr = np.row_stack([carr,np.full((2,4),-1)])

    # above we added the totals row
    # now we need to add the columns for precision and recall

    # create dataframe
    cdf = pd.DataFrame(data = carr,
                      index = [f'ValYear{yr}' for yr in years]+['Mean','StdE'],
                      columns = ['ActPred_00', 'ActPred_01', 
                                 'ActPred_10', 'ActPred_11']
                      )

    cdf['Precision'] = cdf.ActPred_11 / (cdf.ActPred_01 + cdf.ActPred_11)
    cdf['Recall'] = cdf.ActPred_11 / (cdf.ActPred_10 + cdf.ActPred_11)
    cdf['F1'] = 2*cdf.Precision*cdf.Recall / (cdf.Precision + cdf.Recall)
    for col in ['Precision','Recall','F1']:
        cdf.at['Mean',col] = np.mean(cdf.loc[:'ValYear2022',col])
        cdf.at['StdE',col] = np.std(cdf.loc[:'ValYear2022',col])
    
    
    param_strings = [f'# model_name: {model_name}',
                     f'# model: {model}',
                      f'# training_sample_size: {training_sample_size}',
                      f'# validation_sample_size: {validation_sample_size}',
                      f'# tile: {tile}',
                      f'# scheme_name: {scheme_name}',
                      f'# crop_of_interest_id: {crop_of_interest_id}',
                      f'# in_season: {in_season}']
    comment = '\n'.join(param_strings) + '\n' 
    with open(f'../data/results/{csv_name}', 'a') as f:
        f.write(comment)
        cdf.to_csv(f)
    
    print(f'Find results in ../data/results/{csv_name}')
    
    return f'Find results in ../data/results/{csv_name}'

## Run models sequentially

This works, but it takes a long time to churn through models.

In [7]:
models_bank = create_models_bank()
training_sample_size = 0.001
validation_sample_size = 0.001

for tile_coiid in [('10SFH',75),('15TVG',1)]:
    for scheme_name in ['14day','5day']:
        for in_season in [160, 230, None]:
            for three_digit in models_bank.keys():

                ## Parameters dictionary p RANDOM FOREST
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'RF' + three_digit,
                'model': RandomForestClassifier(**models_bank[three_digit]),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season
                }

                #fit_predict_report(**p) # run with the above parameters
                fit_predict_report(**p)
                
                ## Parameters dictionary p EXTRA TREES
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'ET' + three_digit,
                'model': ExtraTreesClassifier(**models_bank[three_digit]),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season
                }

                fit_predict_report(**p)

-- Process for ET042_0.001_0.001_10SFH_14day_75_None.csv --
Starting a fold...
> Assembling the datasets
> Fitting the model on the training set
> Predicting on the validation set
> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets
> Fitting the model on the training set
> Predicting on the validation set
> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets


KeyboardInterrupt: 

## Dask version

Still troubleshooting memory issues on MGHPCC. Given enough memory resources this should run.

In [ ]:
import dask
from dask.distributed import print
from dask.distributed import Client

In [ ]:
client = Client(memory_limit='256GiB')
client

In [ ]:
client.close()

In [ ]:
client = Client(memory_limit='128GiB',
                threads_per_worker=1,
               )

In [ ]:
client

In [ ]:
client.run(trim_memory)

In [ ]:
#client.close()

In [ ]:
trim_fit_predict_report = dask.delayed(trim_fit_predict_report)

In [ ]:
fit_predict_report = dask.delayed(fit_predict_report)

## Regular parallel version:

In [ ]:
models_bank = create_models_bank()
training_sample_size = 0.001
validation_sample_size = 0.001

r = []
for tile_coiid in [('10SFH',75),('15TVG',1)]:
    for scheme_name in ['14day','5day']:
        for in_season in [160, 230, None]:
            for three_digit in models_bank.keys():

                ## Parameters dictionary p RANDOM FOREST
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'RF' + three_digit,
                'model': RandomForestClassifier(**models_bank[three_digit]),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season
                }

                #fit_predict_report(**p) # run with the above parameters
                r.append(fit_predict_report(**p))
                
                ## Parameters dictionary p EXTRA TREES
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'ET' + three_digit,
                'model': ExtraTreesClassifier(**models_bank[three_digit]),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season
                }

                r.append(fit_predict_report(**p))
                
dask.compute(*r)

## Memory-trimming parallel version:

In [ ]:
models_bank = create_models_bank()
training_sample_size = 0.001
validation_sample_size = 0.001

r = []
for tile_coiid in [('10SFH',75),('15TVG',1)]:
    for scheme_name in ['14day','5day']:
        for in_season in [160, 230, None]:
            for three_digit in models_bank.keys():

                ## Parameters dictionary p RANDOM FOREST
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'RF' + three_digit,
                'model': RandomForestClassifier(**models_bank[three_digit]),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season,
                #'client': client
                }

                #fit_predict_report(**p) # run with the above parameters
                r.append(trim_fit_predict_report(**p))
                
                ## Parameters dictionary p EXTRA TREES
                p = {

                ## SPECIFY MODEL ##
                'model_name': 'ET' + three_digit,
                'model': ExtraTreesClassifier(**models_bank[three_digit]),
                'training_sample_size': training_sample_size,
                'validation_sample_size': validation_sample_size,

                ## SPECIFY TILE AND SCHEME ##
                'tile': tile_coiid[0],
                'years': [2018, 2019, 2020, 2021, 2022],
                'scheme_name': scheme_name,
                'crop_of_interest_id': tile_coiid[1], 
                'in_season': in_season,
                #'client': client
                }

                r.append(trim_fit_predict_report(**p))
                
dask.compute(*r)

## Python Multiprocessing way

In [9]:
import multiprocessing as mp
import os

In [8]:
mp.cpu_count()

56

In [11]:
len(os.sched_getaffinity(0))

56

In [9]:
#pool = mp.Pool(56)

-- Process for model_name_training_sample_size_validation_sample_size_tile_scheme_name_crop_of_interest_id_in_season.csv ---- Process for model_name_training_sample_size_validation_sample_size_tile_scheme_name_crop_of_interest_id_in_season.csv ---- Process for model_name_training_sample_size_validation_sample_size_tile_scheme_name_crop_of_interest_id_in_season.csv ---- Process for model_name_training_sample_size_validation_sample_size_tile_scheme_name_crop_of_interest_id_in_season.csv --



Starting a fold...-- Process for model_name_training_sample_size_validation_sample_size_tile_scheme_name_crop_of_interest_id_in_season.csv --Starting a fold...Starting a fold...Starting a fold...




> Assembling the datasets-- Process for model_name_training_sample_size_validation_sample_size_tile_scheme_name_crop_of_interest_id_in_season.csv --Starting a fold...-- Process for model_name_training_sample_size_validation_sample_size_tile_scheme_name_crop_of_interest_id_in_season.csv --> Assembling th

In [10]:
#fit_predict_report = pool.apply_async(fit_predict_report)

In [ ]:
if __name__=='__main__':
    with mp.Pool(56) as pool:
        manager = mp.Manager()
        
        models_bank = create_models_bank()
        training_sample_size = 0.001
        validation_sample_size = 0.001
        
        for tile_coiid in [('10SFH',75),('15TVG',1)]:
            for scheme_name in ['14day','5day']:
                for in_season in [160, 230, None]:
                    for three_digit in models_bank.keys():

                        ## Parameters dictionary p RANDOM FOREST
                        p = {

                        ## SPECIFY MODEL ##
                        'model_name': 'RF' + three_digit,
                        'model': RandomForestClassifier(**models_bank[three_digit]),
                        'training_sample_size': training_sample_size,
                        'validation_sample_size': validation_sample_size,

                        ## SPECIFY TILE AND SCHEME ##
                        'tile': tile_coiid[0],
                        'years': [2018, 2019, 2020, 2021, 2022],
                        'scheme_name': scheme_name,
                        'crop_of_interest_id': tile_coiid[1], 
                        'in_season': in_season
                        }

                        #fit_predict_report(**p) # run with the above parameters
                        pool.apply_async(fit_predict_report, args=p.values()).get()

                        ## Parameters dictionary p EXTRA TREES
                        p = {

                        ## SPECIFY MODEL ##
                        'model_name': 'ET' + three_digit,
                        'model': ExtraTreesClassifier(**models_bank[three_digit]),
                        'training_sample_size': training_sample_size,
                        'validation_sample_size': validation_sample_size,

                        ## SPECIFY TILE AND SCHEME ##
                        'tile': tile_coiid[0],
                        'years': [2018, 2019, 2020, 2021, 2022],
                        'scheme_name': scheme_name,
                        'crop_of_interest_id': tile_coiid[1], 
                        'in_season': in_season
                        }

                        pool.apply_async(fit_predict_report, args=p.values()).get()

-- Process for ET022_0.001_0.001_10SFH_14day_75_None.csv --
Starting a fold...
> Assembling the datasets
> Fitting the model on the training set
> Predicting on the validation set


/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets
> Fitting the model on the training set
> Predicting on the validation set


/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets
> Fitting the model on the training set
> Predicting on the validation set


...

> Recording performance metrics
Finished a fold.
Find results in ../data/results/ET027_0.001_0.001_10SFH_14day_75_None.csv
-- Process for RF028_0.001_0.001_10SFH_14day_75_None.csv --
Starting a fold...
> Assembling the datasets
> Fitting the model on the training set
> Predicting on the validation set


/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets
> Fitting the model on the training set
> Predicting on the validation set


/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/sklearn/ensemble/_base.py:194: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)


> Recording performance metrics
Finished a fold.
Starting a fold...
> Assembling the datasets
